In [388]:
import cv2
import os
import time
import json
import random
from PIL import Image, ImageFont
import numpy as np
from mss import mss
import serial, time
import binascii
import imageio
import urllib
import requests
import textwrap
import itertools

In [ ]:
cap = cv2.VideoCapture(0)
sct = mss()

In [389]:
arduino = serial.Serial('/dev/cu.SLAB_USBtoUART', 115200, timeout=.1)
src_dev = 4
w, h = 64, 64
monitor = {'top': 0, 'left': 0, 'width': w, 'height': h}

In [414]:
gifurl = 'https://gifs.iloopit.net/resources/1ead57a3-48e4-4fb1-86ea-07527a7c079a/converted.gif'
fname = 'tmp.gif'

## Read the gif from the web, save to the disk
if not os.path.exists(fname):
    imdata = urllib.request.urlopen(gifurl).read()
    imbytes = bytearray(imdata)
    open(fname,"wb+").write(imdata)

gif = imageio.mimread('/Users/dusanklinec/Downloads/511e39e7152f0.gif') #fname
#gif = imageio.mimread(fname) #
num_frames = len(gif)
print("Total {} frames in the gif!".format(num_frames))

imgs = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in gif]
imgs = [cv2.resize(rgb, (64, 64)) for rgb in imgs]
imgs = [cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)[1] for img in imgs]

def dtext(d, font, txt, pos):
    mask = font.getmask(txt)
    d.draw_bitmap(pos, mask, 255)

def genBtcImg2():
    white = (255,)# 255, 255)
    img = np.zeros((64,64,1), np.uint8)
    font = cv2.FONT_HERSHEY_PLAIN
    scale = 0.9
    cve = cv2.LINE_8 #cv2.LINE_AA
    
    cv2.putText(img, time.strftime('%l:%M'), (0, 15), font, scale, white, 1, cve)
    cv2.putText(img, "Kraken", (0, 30), font, scale, white, 1, cve)
    cv2.putText(img, "XBTUSD", (0, 45), font, scale, white, 1, cve)
    cv2.putText(img, "5234", (0, 60), font, scale, white, 1, cve)
    return img

def genBtcImg():
    img = Image.new('L', (64, 64), color=0)
    img_w, img_h = img.size
    font = ImageFont.truetype('/Users/dusanklinec/Downloads/basis33/basis33.ttf', 15)
    #font = ImageFont.truetype('/Users/dusanklinec/Downloads/charybdis/chary___.ttf', 12)
    #font = ImageFont.truetype('/Users/dusanklinec/Downloads/big-pixel-shadowed-demo/Big Pixel demo.otf', 10)
    
    d = Image.core.draw(img.im, 0)
    dtext(d, font, time.strftime('%H:%M'), (0,0))
    dtext(d, font, 'Kraken', (0, 20))
    dtext(d, font, 'XBTUSD', (0, 34))
    dtext(d, font, '%.2f' % (random.randrange(510000, 533333)/100.), (0, 48))
    rgb = np.array(img)
    return rgb

def genTextImage(txt):
    img = Image.new('L', (64, 64), color=0)
    img_w, img_h = img.size
    font = ImageFont.truetype('/Users/dusanklinec/Downloads/basis33/basis33.ttf', 14)
    #font = ImageFont.truetype('/Users/dusanklinec/Downloads/big-pixel-shadowed-demo/Big Pixel demo.otf', 9)
    
    d = Image.core.draw(img.im, 0)
    lines = txt.split('\n')
    chunks = [textwrap.wrap(x, 10) for x in lines]
    chunks = list(itertools.chain.from_iterable(chunks))
        
    for ix, ch in enumerate(chunks):
        dtext(d, font, ch, (0, ix * 13))

    rgb = np.array(img)
    return rgb

def dumpXbm(data, fl):
    fl.write('#define pica_width 64\n')
    fl.write('#define pica_height 64\n')
    fl.write('static char pica_bits[] = {')
    for ix, x in enumerate(data):
        fl.write('0x%02X, ' % x)
    fl.write('};\n')
    
quotes = [
    'Spoiler:\nDaenerys dies in S08E04',
    'Spoiler:\nDaenerys\'es dragons die in S08E04',
    'Spoiler:\nThanos is immortal',
    'Spoiler:\nDr. Strange dies in Endgame',
    'Spoiler:\nEndgame has no post-credits',
    'Spoiler:\nVision is not dead',
    'Don\'t drink and root',
    'Bitcoin is a lie',
    '3rd WC is not flushing',
    'There is no dinner today',
    'Carefully with sage',
    'Uz len tuto flasu som povedal',
    'Mne to neskodi, mne to prospieva',
    'Krasny den na trip',
]

Total 21 frames in the gif!


In [415]:
# imgs = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in gif]
# imgs = [cv2.resize(rgb, (64, 64)) for rgb in imgs]
# imgs = [cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)[1] for img in imgs]


## Display the gif
# i = 0
# while True:
#     cv2.imshow("gif", imgs[i])
#     if cv2.waitKey(100)&0xFF == 27:
#         break
#     i = (i+1)%nums
# cv2.destroyAllWindows()

In [ ]:
cframe = 0
ltx = 0
ldump = 0
linco = 0
cquote = 0
lastbuff = bytearray(512)
while True:
    sinceLastInco = time.time() - linco
    if (time.time() - ldump > 13) or (sinceLastInco > 2 and sinceLastInco < 5 and time.time() - ldump > 5):
        arduino.write('!DUMP'.encode())
        arduino.flush()
        time.sleep(1.5)
        ldump = time.time()
        print('plsdump')
    
    while True:
        rline = arduino.readline()
        if len(rline) == 0:
            break
            
        if rline.startswith(b'!RECV'):
            print('dumped')
            cbuff = rline[5:]
            tst = time.time()
            clen = len(rline) - 5
            while(clen < 512 and time.time() - tst < 5):
                cbuff += arduino.read(512-clen)
                clen = len(cbuff)
            if clen == 512: 
                lastbuff = cbuff
                dumpXbm(cbuff, open('/tmp/last.xbm', 'w+'))
                time.sleep(0.5)
                continue
            else:
                print('Corrupted buff, len: %s, buff: %s' % (clen, binascii.hexlify(cbuff)))
                
        elif rline.startswith(b'!INC'):
            print('.')
            ltx += 1
            linco = time.time()
            
        else:
            print('INC: ', rline)
    
    if time.time() - ltx < 10:
        continue
        
    ltx = time.time()
    
    if src_dev == 0:
        ret, frame = cap.read()
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    elif src_dev == 1:
        rgb = np.array(sct.grab(monitor))
        rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)
    elif src_dev == 2:
        rgb = imgs[cframe]
        cframe = (cframe + 15) % num_frames
    elif src_dev == 3:
        rgb = genBtcImg()
    elif src_dev == 4:
        rgb = genTextImage(quotes[random.randrange(0, len(quotes))])
        cquote += 1
    else:
        raise ValueError('wtf')
    obuff = rgb
    
    #obuff = cv2.adaptiveThreshold(obuff, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 15, 1)
    obuff = cv2.resize(obuff, (64, 64))#, interpolation = cv2.INTER_NEAREST)
    ret, obuff = cv2.threshold(obuff, 128, 255, cv2.THRESH_BINARY)
    #obuff = cv2.adaptiveThreshold(obuff, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 5, 1)
    
    nobuff = bytearray(int(64*64/8))  # bitmap picture, 1 Byte = 8 pixels
    cbit = 0
    for i in range(64*64):
        bout = obuff[i//64][i%64] > 0
        if bout:
            nobuff[i//8] |= 1 << (i%8)
            
    #print(binascii.hexlify(nobuff))

    sbuff = bytearray(65)
    off = 0
    for r in range(8):
        sbuff[0] = r
        for i in range(64):
            sbuff[1+i] = nobuff[off+i]
            #if r & 1 == 0:
            #    sbuff[1+i] ^= 0xff
        off += 64

        ln = arduino.write(sbuff)
        arduino.flush()
        #print('%s, %s' % (ln, binascii.hexlify(sbuff)))
        time.sleep(0.15)
   
    cv2.imshow('frame', obuff)
    cv2.waitKey(1)
    print('n')
    #time.sleep(2)
    #break
    
    # TODO: collect packets, log, RSSI, retransmit over BT to phone...


plsdump
dumped
Corrupted buff, len: 519, buff: b'fffffffffffdfffffffffffffff8ffffffffffff7ff8ffffffffffff7ff8ffffffffffff3ff8ffffffffffff3ff8ffffffffffff1ff8ffffffffffff0ff0ffff00000000f83f000000000000fc7f000000000000fcff000000000000fcff000000000000feff030000000000feff070000000000fcff0f0000000000feff070000000000ffff070000000080ffff0100000000c0ffff0000000000e0ff3f0000000000e0ff1f0000000000f0ff1f0000000000f8ff1f0000000000fcff1f0000000000feff1f0000000000feff3f0000000000ffff1f0000000080ffff1f00000000c0ffff1f00000000e0ffff1f00000000e0ffff1f00000000f0ffff0f00000000f8ffff0f00000000fcffff0700000000fcffff0300000000feffff0300000000feffff0300000000feffff0300000000ffffff0300000000ffffff030000ffff000000fcffffffff000000fcffffffff000000f8ffffffff000000fcffffff7f000000fcffffffff000000f8ffffffff000000f8ffffffff000020f8ffffff7f000020f8ffffff3f000030f8ffffff3f000070f0ffffff1f000060c0ffffff1f00004000ffffff1f06008000ffffff0f1e008000feffff1f3e008000ffffff0ffeabf7ebffffff1ffefffdffffffff1ffc1ffcffffffff1ff80

In [ ]:
cv2.destroyAllWindows()

In [ ]:
#w, h = 200, 200
#monitor = {'top': 0, 'left': 0, 'width': w, 'height': h}
#img = Image.frombytes('RGB', (w,h), sct.grab(monitor).rgb)
#cv2.imshow('frame', np.array(img))
cv2.imshow('frame', np.array(sct.grab(monitor)))
if cv2.waitKey(0) & 0xFF == ord('q'):
    cv2.destroyAllWindows()
    

In [ ]:
ret, frame = cap.read()
rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)
print(rgb[0][0])

In [ ]:
cap.release()

In [ ]:
while(True):
#for i in range(2):
    ret, frame = cap.read()
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)
    W = 200
    roff = random.randrange((720-W)*(720-W))
    for r in range(W):
        for q in range(W):
            rgb[(roff//720)+r][(roff%720)+q] = [0x0, 0xff, 0x00, 0xff]
    
    ob = cv2.resize(rgb, (64,64))
    cv2.imshow('image', random.randrange(3,10)*rgb)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        out = cv2.imwrite('capture.jpg', frame)
        break

cap.release()
cv2.destroyAllWindows()
   


In [ ]:
import serial, time
arduino = serial.Serial('/dev/cu.SLAB_USBtoUART', 115200, timeout=.1)

In [ ]:
arduino = serial.Serial('/dev/cu.SLAB_USBtoUART', 115200, timeout=.1)
while True:
    line = arduino.readline().strip()
    if len(line) == 0:
        continue
    print(line)